In [1]:
import sys
sys.path.append('..')
import time
from astropy.table import Table
import pyfits as fits
import numpy as np
import matplotlib.pyplot as plt
from os.path import expanduser
import random
from sklearn.utils import shuffle
import re
import datetime
from sklearn.model_selection import train_test_split
from deeplens.resnet_classifier import deeplens_classifier

home = expanduser("~")

Using Theano backend.


In [18]:
def scramble_lens(lens, model_num):
    index = np.load('lens_file_indicies.npy')
    x_lens = np.zeros((lens,3,101,101))
    y_lens = np.ones((lens,1))
    
    decalsA = 52
    decalsB = 22
    mzls = 7
    groupdecals = 30
    groupmzls = 12
    
    num_index = np.array([])
    for i in index:
        num_index = np.concatenate((num_index,np.asarray((int(filter(str.isdigit, i)))).reshape(-1,)))
        
    index, none, num_index, none = train_test_split(index, num_index, test_size=1, shuffle = True)
    
    print index
    print num_index
    
    # loads lens images 
    for i, (idx, num_idx) in enumerate(zip(index,num_index)):
        if idx[-1] == 'A':
#             print num_idx, idx
            x_lens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Train/fits_images/lens_cutout_gradeA_{:06d}.fits'.format(int(num_idx)))
        elif idx[-1] == 'B':
#             print num_idx, idx
            x_lens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Train/fits_images/lens_cutout_gradeB_{:06d}.fits'.format(int(num_idx)))
        elif idx[-1] == 'S':
#             print num_idx, idx
            x_lens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Train/fits_images/lens_cutout_MzLS_{:06d}.fits'.format(int(num_idx)))
        elif 'DECals_group' in idx:
#             print num_idx, idx
            x_lens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Train/fits_images/lens_cutout_group_DECals_{:06d}.fits'.format(int(num_idx)))
        elif 'MzLS_group' in idx:
#             print num_idx, idx
            x_lens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Train/fits_images/lens_cutout_group_MzLS_{:06d}.fits'.format(int(num_idx)))
    
    
#     np.save('/Users/Chris/Desktop/DECals_set/Trained_Models/model_{}/x_y_sets/shuffled_index_lens.npy'.format(model_num), index)
#     print 'shuffled indices saved.'
    
    x_train = x_lens[:88]
    xval = x_lens[88:]
    y_train = y_lens[:88]
    yval = y_lens[88:]
    
    print 'x_lens_train shape:',x_train.shape
    print 'y_lens_train shape: ', y_train.shape
    print 'confirm number of lenses (train): ', np.sum(y_train)
    print 'x_lens_val shape:',xval.shape
    print 'y_lens_val shape:',yval.shape
    print 'confirm number of lenses(val): ', np.sum(yval)
    
    
    return x_train, xval, y_train, yval

In [19]:
x_lens_train, x_lens_val, y_lens_train, y_lens_val = scramble_lens(123,5)

['27DECals_group' '28A' '7MzLS' '54A' '15B' '48A' '8MzLS' '1MzLS' '10B'
 '21A' '6B' '42A' '32A' '18B' '9B' '46A' '55A' '15MzLS_group' '51A' '8A'
 '14MzLS_group' '24A' '8MzLS_group' '10MzLS_group' '20B' '13DECals_group'
 '65A' '11A' '37A' '19B' '1MzLS_group' '69A' '14A' '34A' '7MzLS_group'
 '35A' '66A' '16A' '61A' '34DECals_group' '32DECals_group' '29DECals_group'
 '16B' '45A' '53A' '17B' '35DECals_group' '11DECals_group' '28DECals_group'
 '7A' '26B' '30DECals_group' '0A' '10A' '22DECals_group' '3MzLS' '25B' '4A'
 '12B' '13B' '5B' '3DECals_group' '5A' '43A' '52A' '15DECals_group' '39A'
 '36A' '21DECals_group' '38A' '56A' '25DECals_group' '5MzLS_group' '0MzLS'
 '41A' '9DECals_group' '6DECals_group' '13A' '27B' '2DECals_group' '9A'
 '4B' '2MzLS' '20DECals_group' '47A' '10DECals_group' '19A' '31A' '64A'
 '5MzLS' '4MzLS_group' '33A' '60A' '58A' '59A' '17DECals_group' '29A'
 '37DECals_group' '18DECals_group' '12MzLS_group' '0MzLS_group' '11B' '50A'
 '1B' '22B' '63A' '33DECals_group' '5DECals

In [3]:
def scramble_nonlens(nonlens, model_num):
    index = np.load('selected_train_images.npy')
    x_nonlens = np.zeros((nonlens,3,101,101))
    y_nonlens = np.zeros((nonlens,1))
    print index[:10]
    index, none, none, none = train_test_split(index,np.zeros((index.shape[0])), test_size=1, shuffle = True)
    
    print index[:10]
    for i, idx in enumerate(index):
        x_nonlens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Non-Lenses/images/cutout_{:06d}.fits'.format(int(idx)))
    
    np.save('/Users/Chris/Desktop/DECals_set/Trained_Models/model_{}/x_y_sets/shuffled_index_nonlens.npy'.format(model_num), index)
    print 'shuffled indices saved.'
    
    x_train = x_nonlens[:180]
    xval = x_nonlens[180:]
    y_train = y_nonlens[:180]
    yval = y_nonlens[180:]
    
    print 'x_nonlens_train shape:',x_train.shape
    print 'y_nonlens_train shape: ', y_train.shape
    print 'confirm number of nonlenses (train): ', np.sum(y_train)
    print 'x_nonlens_val shape:',xval.shape
    print 'y_nonlens_val shape:',yval.shape
    print 'confirm number of nonlenses(val): ', np.sum(yval)
    
    
    return x_train, xval, y_train, yval

In [59]:
x_nonlens_train, x_nonlens_val, y_nonlens_train, y_nonlens_val =scramble_nonlens(200,5)

[  0.   1.   2.   3.   4.  22.  24.  26.  27.  31.]
[  674.   120.   923.    34.   858.   581.  1431.   504.   149.    24.]
shuffled indices saved.
x_nonlens_train shape: (180, 3, 101, 101)
y_nonlens_train shape:  (180, 1)
confirm number of nonlenses (train):  0.0
x_nonlens_val shape: (20, 3, 101, 101)
y_nonlens_val shape: (20, 1)
confirm number of nonlenses(val):  0.0


In [60]:
x_train, y_train, xval, yval = np.concatenate((x_lens_train,x_nonlens_train)), np.concatenate((y_lens_train,y_nonlens_train)), np.concatenate((x_lens_val,x_nonlens_val)), np.concatenate((y_lens_val,y_nonlens_val))

In [61]:
print 'x_train shape:',x_train.shape
print 'y_train shape: ', y_train.shape
print 'confirm number of lenses (train): ', np.sum(y_train)
print 'xval shape:',xval.shape
print 'yval shape:',yval.shape
print 'confirm number of lenses(val): ', np.sum(yval)

x_train shape: (240, 3, 101, 101)
y_train shape:  (240, 1)
confirm number of lenses (train):  60.0
xval shape: (41, 3, 101, 101)
yval shape: (41, 1)
confirm number of lenses(val):  21.0


In [12]:
index = np.load('lens_file_indicies.npy')
print index

['0A' '3A' '4A' '5A' '7A' '8A' '9A' '10A' '11A' '12A' '13A' '14A' '16A'
 '18A' '19A' '21A' '23A' '24A' '28A' '29A' '31A' '32A' '33A' '34A' '35A'
 '36A' '37A' '38A' '39A' '41A' '42A' '43A' '45A' '46A' '47A' '48A' '50A'
 '51A' '52A' '53A' '54A' '55A' '56A' '58A' '59A' '60A' '61A' '63A' '64A'
 '65A' '66A' '69A' '1B' '4B' '5B' '6B' '7B' '9B' '10B' '11B' '12B' '13B'
 '15B' '16B' '17B' '18B' '19B' '20B' '22B' '23B' '24B' '25B' '26B' '27B'
 '0MzLS' '1MzLS' '2MzLS' '3MzLS' '5MzLS' '7MzLS' '8MzLS' '2DECals_group'
 '3DECals_group' '4DECals_group' '5DECals_group' '6DECals_group'
 '7DECals_group' '8DECals_group' '9DECals_group' '10DECals_group'
 '11DECals_group' '13DECals_group' '15DECals_group' '17DECals_group'
 '18DECals_group' '19DECals_group' '20DECals_group' '21DECals_group'
 '22DECals_group' '25DECals_group' '27DECals_group' '28DECals_group'
 '29DECals_group' '30DECals_group' '32DECals_group' '33DECals_group'
 '34DECals_group' '35DECals_group' '36DECals_group' '37DECals_group'
 '38DECals_gro

In [17]:
lens = 123

x_lens = np.zeros((lens,3,101,101))
y_lens = np.ones((lens,1))

decalsA = 52
decalsB = 22
mzls = 7
groupdecals = 30
groupmzls = 12

num_index = np.array([])
for i in index:
    num_index = np.concatenate((num_index,np.asarray((int(filter(str.isdigit, i)))).reshape(-1,)))

# index, none, num_index, none = train_test_split(index, num_index, test_size=1, shuffle = True)

print index
print num_index

# loads lens images 
for i, (idx, num_idx) in enumerate(zip(index,num_index)):
    if idx[-1] == 'A':
        print num_idx, idx
        x_lens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Train/fits_images/lens_cutout_gradeA_{:06d}.fits'.format(int(num_idx)))
    elif idx[-1] == 'B':
        print num_idx, idx
        x_lens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Train/fits_images/lens_cutout_gradeB_{:06d}.fits'.format(int(num_idx)))
    elif idx[-1] == 'S':
        print num_idx, idx
        x_lens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Train/fits_images/lens_cutout_MzLS_{:06d}.fits'.format(int(num_idx)))
    elif 'DECals_group' in idx:
        print num_idx, idx
        x_lens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Train/fits_images/lens_cutout_group_DECals_{:06d}.fits'.format(int(num_idx)))
    elif 'MzLS_group' in idx:
        print num_idx, idx
        x_lens[i] = fits.getdata('/Users/Chris/Desktop/DECals_set/Train/fits_images/lens_cutout_group_MzLS_{:06d}.fits'.format(int(num_idx)))
print num_index.shape
print index.shape
    
# np.save('/Users/Chris/Desktop/DECals_set/Trained_Models/model_{}/x_y_sets/shuffled_index_lens.npy'.format(model_num), index)
# print 'shuffled indices saved.'

# x_train = x_lens[:60]
# xval = x_lens[60:]
# y_train = y_lens[:60]
# yval = y_lens[60:]

# print 'x_lens_train shape:',x_train.shape
# print 'y_lens_train shape: ', y_train.shape
# print 'confirm number of lenses (train): ', np.sum(y_train)
# print 'x_lens_val shape:',xval.shape
# print 'y_lens_val shape:',yval.shape
# print 'confirm number of lenses(val): ', np.sum(yval)
    

['0A' '3A' '4A' '5A' '7A' '8A' '9A' '10A' '11A' '12A' '13A' '14A' '16A'
 '18A' '19A' '21A' '23A' '24A' '28A' '29A' '31A' '32A' '33A' '34A' '35A'
 '36A' '37A' '38A' '39A' '41A' '42A' '43A' '45A' '46A' '47A' '48A' '50A'
 '51A' '52A' '53A' '54A' '55A' '56A' '58A' '59A' '60A' '61A' '63A' '64A'
 '65A' '66A' '69A' '1B' '4B' '5B' '6B' '7B' '9B' '10B' '11B' '12B' '13B'
 '15B' '16B' '17B' '18B' '19B' '20B' '22B' '23B' '24B' '25B' '26B' '27B'
 '0MzLS' '1MzLS' '2MzLS' '3MzLS' '5MzLS' '7MzLS' '8MzLS' '2DECals_group'
 '3DECals_group' '4DECals_group' '5DECals_group' '6DECals_group'
 '7DECals_group' '8DECals_group' '9DECals_group' '10DECals_group'
 '11DECals_group' '13DECals_group' '15DECals_group' '17DECals_group'
 '18DECals_group' '19DECals_group' '20DECals_group' '21DECals_group'
 '22DECals_group' '25DECals_group' '27DECals_group' '28DECals_group'
 '29DECals_group' '30DECals_group' '32DECals_group' '33DECals_group'
 '34DECals_group' '35DECals_group' '36DECals_group' '37DECals_group'
 '38DECals_gro